<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with Multi-Agent Personas

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.45" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.45"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.Agents.Core, 1.45.0

### Step 2 - Creating a Multi-Agent Decision Personas 

> "The many are smarter than the few when it comes to solving problems, predicting outcomes, and innovating solutions, as long as there is diversity of opinion, independence, and decentralization."  
>
> -- <cite>James Surowiecki (Author "The Wisdom of Crowds")</cite> 

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other mechanisms with prompts. **Creating multiple personas (experts) that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple chat agents to interact in various ways.** 

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand. In the output notice key steps:
* Each expert (Dwight and Ben) make their recommendation, based on their decision persona
* The final recommendation is made based on the synthesis of the agent recommendations  
* This multi-agent interaction to converge on a single decision mimic the self-consistency GenAI pattern 

Note: The execution below can take the some time. Use GPT-4o-mini to optimize performance speed of the execution.  

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Threading;

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var assistantDwightEisenhower = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DwightEisenhower", // Dwight Eisenhower persona
    Description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "BenFranklin", // Ben Franklin persona
    Description = "A chat bot that replies to the message with recommending the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DecisionMaker",
    Description = "A chat bot that approves the final decision.",
    Instructions = """
    You are a Decision Intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand. Don't waste time with chit chat.
    Consider suggestions when refining an idea. Use the words final decision to approve the final decision.
    """
};

// Create a chat for agent interaction.
AgentGroupChat chat =
    new(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    }
            }
    };

// Invoke chat and display messages.
ChatMessageContent input = new(AuthorRole.User, """
Decision: Buy a house for a rental property or invest in Bitcoin? 
Constraint: You want to optimize for long-term financial growth, but also mitigate risk. 
""");
chat.AddChatMessage(input);
Console.WriteLine(input);
Console.WriteLine();

await foreach (ChatMessageContent response in chat.InvokeAsync())
{
    Console.WriteLine(response);
}

Console.WriteLine($"\n[IS COMPLETED: {chat.IsComplete}]");

Decision: Buy a house for a rental property or invest in Bitcoin? 
Constraint: You want to optimize for long-term financial growth, but also mitigate risk. 

This decision has significant financial and strategic implications. Here's a structured approach to analyzing your options using Dwight Eisenhower's prioritization matrix and systems thinking principles, helping you sort out urgency and importance, weigh risks, and align your choice with long-term growth goals. 

---

### I. **Key Factors and Context Analysis**
1. **Objective**: Long-term financial growth while mitigating risks.
2. **Options**: Buy a house for rental income (real estate) **vs.** invest in Bitcoin (cryptocurrency).
3. **Constraints**:
   - You want growth over time (indicates preference for assets with appreciation potential).
   - You want to mitigate risk (indicates a need to analyze volatility, security, and reliability).

---

### II. **Eisenhower Matrix for Decision Prioritization**
Let’s categorize tasks into